# Celcius to Farenheit equation

$$ f = c \times 1.8 + 32 $$

# Import TensorFlow 2.x.

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

import numpy as np
np.random.seed(7)

import matplotlib.pyplot as plot

print(tf.__version__)

# Compute Farenheit using Celcius.

In [0]:
def celsius_to_fahrenheit(celsius_value):
  fahrenheit_value = celsius_value * 1.8 + 32
  return(fahrenheit_value)

# Generate dataset for converting Celcius to Farenheit.

In [0]:
def generate_dataset(number_of_samples=100):

  celsius_values = []
  fahrenheit_values = []

  value_range = number_of_samples

  for sample in range(number_of_samples):    
    celsius_value = np.random.randint(-1*value_range, +1*value_range)
    fahrenheit_value = celsius_to_fahrenheit(celsius_value)

    celsius_values.append(celsius_value)
    fahrenheit_values.append(fahrenheit_value)

  return(celsius_values, fahrenheit_values)

In [0]:
number_of_samples = 100

In [0]:
celsius_values, fahrenheit_values = generate_dataset(number_of_samples)
for index, celsius_value in enumerate(celsius_values):
  print("{} degrees Celsius = {} degrees Fahrenheit".format(celsius_value, fahrenheit_values[index]))

# Create artificial neural network model.

In [0]:
model = models.Sequential(name='model')
model.add(layers.Dense(units=1, input_shape=[1], name='dense_layer'))
model.summary()

# Create the optimizer.

In [0]:
from tensorflow.keras.optimizers import Adam

learning_rate = 0.2
optimizer = Adam(learning_rate=learning_rate)

# Compute the mean squared loss.

In [0]:
def compute_mean_squared_loss(y_true, y_pred):  
  y_true = tf.cast(y_true, y_pred.dtype)
  loss = tf.math.reduce_mean(tf.math.squared_difference (y_true, y_pred), axis=-1)
  print('y_true -', y_true.numpy())
  print('y_pred -', y_pred.numpy())
  print('MSE -', loss.numpy())
  return(loss)

In [0]:
maximum_iterations = 300

In [0]:
iteration = 0
while iteration < maximum_iterations:
  iteration = iteration + 1
  print('******************************************************************************************')
  print('iteration', iteration)  
  print("model weights - {}".format(model.get_layer(index=-1).get_weights()))  

  index = np.random.randint(0, number_of_samples)
  input_celsius_sample, true_fahrenheit_sample = celsius_values[index], fahrenheit_values[index]  
  
  print('celsius value -', input_celsius_sample)
  print('fahrenheit value -', true_fahrenheit_sample)  

  with tf.GradientTape(persistent=True, watch_accessed_variables=False) as tape:
    tape.watch(model.trainable_variables)

    input_celsius = tf.expand_dims(input_celsius_sample, -1)
    true_fahrenheit = tf.expand_dims(true_fahrenheit_sample, -1)
  
    predicted_fahrenheit = model(input_celsius, training=True)
    loss = compute_mean_squared_loss(true_fahrenheit, predicted_fahrenheit[0])

  gradients = tape.gradient(loss, model.trainable_variables)  
  print('gradients -', gradients[0].numpy(), gradients[1].numpy())

  optimizer.apply_gradients(zip(gradients, model.trainable_variables))